In [8]:
import pydub
from pydub.playback import play
from skimage import io,color
import numpy as np
import pandas as pd
image_path = "/home/billbrod/Pictures/WFB Brussels.jpg"
import melopy

In [2]:
img = io.imread(image_path)
#We want to use the L*a*b color space instead of RGB because it approximates human perceptual space
lab = color.rgb2lab(img)

In [3]:
def color_df(arr,col,color_name):
    df=pd.DataFrame(arr[:,:,col])
    df['x'] = df.index
    df = pd.melt(df,id_vars='x',var_name='y',value_name=color_name)
    return df

In [4]:
L,a,b = [color_df(i,j,k) for i,j,k in zip([lab,lab,lab],[0,1,2],['L','a','b'])]
df = pd.merge(L,pd.merge(a,b))

Based on http://stackoverflow.com/a/29906003, $L \in [0,100], a,b \in [-128,127]$. We need to know our min and max in order to find our mappable range.

Our color space is three dimensional ($L,a,b$) and for sound to be three dimensional, we're going to use octave ($L$), tone ($a$) and volume ($b$). Make those re-mappable, but we'll see how that works.

According to [Wikipedia](https://en.wikipedia.org/wiki/Hearing_range#Humans), humans can hear from 20Hz up to 20kHz and are most sensitive to frequencies between 2kHz and 5kHz.

Pydub can be used to generate sounds really well, but it's woefully under-documented. See [this gist](https://gist.github.com/jiaaro/339df443b005e12d6c2a) for a reasonable example (that `output` object can be played, no need to export it!)

Melopy.utilities has some useful tools to convert between notes and frequencies.

If we need to slice up the color space into cubes of just-noticeable-differences, use `skimage.color.deltaE_ciede2000`. It seems that a delta_e of 1 is the "just noticeable difference". Having trouble getting this to work, may need to modify slightly.